In [1]:
import os, glob, random
import pandas as pd
import SimpleITK as sitk
import numpy as np
from scipy import stats  

def get_image_path_by_id(patient_id, image_dir):
    image_order  = patient_id
    file_name = [os.path.relpath(os.path.join(image_dir, x)) \
                    for x in os.listdir(image_dir) \
                    if os.path.isfile(os.path.join(image_dir, x)) and patient_id in x][0] 
    return file_name

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
csv_path = 'MRI_volume_internal_test331.csv'

df = pd.read_csv(csv_path)

df_compare = df
print(df_compare.shape)
df_compare.head()

(331, 10)


,Data_set,CT_id,volume1_gd,volume1_ai,volume2_gd,volume2_ai,volume3_gd,volume3_ai,volume4_gd,volume4_ai
0,MRI515,0299_T2,170.11519,172.20813,194.23134,191.95846,166.86214,166.70588,188.93270,189.35886
1,MRI515,0537_T2,201.85218,201.29868,192.23131,193.20245,177.24146,179.00260,207.60860,209.52070
2,MRI515,0450_T1,243.65192,242.78687,96.87088,95.30644,96.70063,96.84787,218.77276,216.18684
3,MRI515,0019_T2,378.97104,373.09953,155.45229,154.73026,143.26801,141.63341,359.64668,356.35743
4,MRI515,0387_T2,175.05756,175.03249,109.21884,109.46109,114.64449,114.17251,178.15256,178.43240


In [4]:
import numpy as np
from scipy.stats import ks_1samp, norm, skew, kurtosis
from statsmodels.stats.weightstats import ttost_paired

# Parameters
equivalence_margin_percent = 0.05
n_bootstrap = 10000
confidence_level = 0.95

def run_tost(ai, man, margin_pct, skew_thresh=0.5, kurt_thresh=1.0):
    """
    Perform TOST or Bootstrap TOST on paired data ai vs. man.
    """
    delta = man.mean() * margin_pct
    low, upp = -delta, delta
    diff = ai - man

    # Normality & skewness/kurtosis evaluation
    z = (diff - diff.mean()) / diff.std(ddof=1)
    ks_stat, ks_p = ks_1samp(z, cdf=norm.cdf)
    s = skew(diff)
    k = kurtosis(diff, fisher=True)

    print(f"[KS Normality] p = {ks_p:.4f} | skewness = {s:.3f} | excess kurtosis = {k:.3f}")
    use_tost = (ks_p >= 0.05) or (abs(s) < skew_thresh and abs(k) < kurt_thresh)
    print("→ Using", "TOST" if use_tost else "Bootstrap TOST")
    print(f"Sample size: {len(diff)} | Mean difference: {diff.mean():.4f} | ±delta: {delta:.4f}")

    if use_tost:
        p_tost, (t1, p1, _), (t2, p2, _) = ttost_paired(ai, man, low, upp)
        print(f"TOST: lower t={t1:.4f}, p={p1:.4f}; upper t={t2:.4f}, p={p2:.4f}")
        print(f"pTOST = {p_tost:.4f}", "✅ Equivalent" if p_tost < 0.05 else "❌ Not Equivalent")
    else:
        # Nonparametric (bootstrap) TOST p-value calculation
        boots = np.random.choice(diff, size=(n_bootstrap, len(diff)), replace=True).mean(axis=1)
        p_lo = np.mean(boots <= -delta)
        p_hi = np.mean(boots >=  delta)
        p_boot = max(p_lo, p_hi)
        print(f"Bootstrap pTOST = {p_boot:.4f}", 
              "✅ Equivalent" if p_boot < 0.05 else "❌ Not Equivalent")

# Loop over each volume class (volume1 to volume4)
for i in range(1, 5):
    gd_col = f"volume{i}_gd"
    ai_col = f"volume{i}_ai"
    # Extract AI vs. ground-truth arrays
    ai = df[ai_col].values
    man = df[gd_col].values

    print(f"\n==== Class: Volume{i} ====")
    if len(ai) == len(man) and len(ai) > 0:
        run_tost(ai, man, equivalence_margin_percent)
    else:
        print("❌ Data not properly paired or missing.")



==== Class: Volume1 ====
[KS 正态性] p = 0.2114 | skew = 0.458 | excess kurtosis = 2.291
→ Using TOST
Sample size: 331 | Mean diff: 0.6754 | ±delta: 12.5403
TOST: lower t=54.9682, p=0.0000; upper t=-49.3495, p=0.0000
pTOST = 0.0000 ✅ Equivalent

==== Class: Volume2 ====
[KS 正态性] p = 0.0077 | skew = 0.192 | excess kurtosis = 3.423
→ Using Bootstrap TOST
Sample size: 331 | Mean diff: 0.0889 | ±delta: 7.1395
Bootstrap pTOST = 0.0000 ✅ Equivalent

==== Class: Volume3 ====
[KS 正态性] p = 0.0180 | skew = 1.067 | excess kurtosis = 7.267
→ Using Bootstrap TOST
Sample size: 331 | Mean diff: 0.2287 | ±delta: 7.1150
Bootstrap pTOST = 0.0000 ✅ Equivalent

==== Class: Volume4 ====
[KS 正态性] p = 0.1123 | skew = -0.630 | excess kurtosis = 3.376
→ Using TOST
Sample size: 331 | Mean diff: -0.0247 | ±delta: 12.9829
TOST: lower t=59.6704, p=0.0000; upper t=-59.8977, p=0.0000
pTOST = 0.0000 ✅ Equivalent
